# Load Data : 

In [1]:
from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv

spark = SparkSession.builder \
    .appName("BinancePipeline") \
    .getOrCreate()



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/28 09:52:56 WARN Utils: Your hostname, MAE, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/28 09:52:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/28 09:52:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

# bronze_path = '/mnt/c/Users/user/Desktop/Quant-AI-Project/ml/data/btc_minute_data.parquet'
bronze_path=os.getenv("bronze_path_env")
df_bronze = spark.read.parquet(bronze_path)

df_bronze.show(5)
df_bronze.printSchema()

+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+
|          open_time|    open|    high|     low|   close|  volume|          close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------------------+--------+--------+--------+--------+--------+--------------------+------------------+----------------+---------------------+----------------------+------+
|2026-01-25 07:46:00|88722.27|88722.27|88722.25|88722.26|  4.3435|2026-01-25 07:46:...|    385365.1276877|             436|              0.16568|         14699.5040388|     0|
|2026-01-25 07:47:00|88722.25|88722.26|88722.25|88722.25| 0.64802|2026-01-25 07:47:...|      57493.793677|             117|               0.1232|          10930.582432|     0|
|2026-01-25 07:48:00|88722.25|88722.26|88722.25|88722.25| 0.70972|2026-01-25 07:48:...|     62967.9582888|             1

In [3]:
df_bronze.summary().show()

26/01/28 09:53:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+---------------------+----------------------+------+
|summary|             open|              high|              low|             close|            volume|quote_asset_volume|  number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+------------------+---------------------+----------------------+------+
|  count|              600|               600|              600|               600|               600|               600|               600|                  600|                   600|   600|
|   mean|88512.13728333337| 88527.95423333347|88494.34791666668| 88510.93191666667| 8.130447683333326| 717858.2180273327|2254.1066666666666|    3.906716066666667|     345014.8334012689|   0.0|
| stddev|296.2697389161398|286.1310

In [4]:
df_bronze.printSchema()

root
 |-- open_time: timestamp_ntz (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)
 |-- close_time: timestamp_ntz (nullable = true)
 |-- quote_asset_volume: double (nullable = true)
 |-- number_of_trades: long (nullable = true)
 |-- taker_buy_base_volume: double (nullable = true)
 |-- taker_buy_quote_volume: double (nullable = true)
 |-- ignore: string (nullable = true)



In [5]:
# Check for nulls
from pyspark.sql.functions import col, sum as spark_sum, when

null_counts = df_bronze.select([
    spark_sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) 
    for c in df_bronze.columns
])
null_counts.show()

+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+
|open_time|open|high|low|close|volume|close_time|quote_asset_volume|number_of_trades|taker_buy_base_volume|taker_buy_quote_volume|ignore|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+
|        0|   0|   0|  0|    0|     0|         0|                 0|               0|                    0|                     0|     0|
+---------+----+----+---+-----+------+----------+------------------+----------------+---------------------+----------------------+------+



In [6]:
print("Déduplication...")
rows_before = df_bronze.count()
df_bronze = df_bronze.dropDuplicates(["open_time"])
duplicates = rows_before - df_bronze.count()
print(f" {duplicates} doublons supprimés")

Déduplication...
 0 doublons supprimés
